In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install stanza; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza
# Import stanza
import stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=cde039282661729a23e072c315f4c5212ff8df618200320f66ca1c81666e4d74
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji


In [ ]:
# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)
# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

INFO:stanza:Installing CoreNLP package into ./corenlp


In [ ]:
# Import client module
from stanza.server import CoreNLPClient

In [ ]:
# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
client = CoreNLPClient(
    annotators=['tokenize','ssplit', 'pos', 'parse', 'lemma', 'ner', 'sentiment'],
    memory='4G',
    endpoint='http://localhost:9001',
    be_quiet=True)
print(client)

# Start the background server and wait for some time
# Note that in practice this is totally optional, as by default the server will be started when the first annotation is performed
client.start()
import time; time.sleep(10)

INFO:stanza:Writing properties to tmp file: corenlp_server-ddcc0abb6d5e4609.props
INFO:stanza:Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-ddcc0abb6d5e4609.props -annotators tokenize,ssplit,pos,parse,lemma,ner,sentiment -preload -outputFormat serialized


In [ ]:
# Print background processes and look for java
# You should be able to see a StanfordCoreNLPServer java process running in the background
!ps -o pid,cmd | grep java

    793 java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-ddcc0abb6d5e4609.props -annotators tokenize,ssplit,pos,parse,lemma,ner,sentiment -preload -outputFormat serialized
    862 /bin/bash -c ps -o pid,cmd | grep java
    864 grep java


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Redback_A/chatgpt.csv")
data = data.drop("Unnamed: 0", axis=1)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219294 entries, 0 to 219293
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   tweets  219294 non-null  object
 1   labels  219294 non-null  object
dtypes: object(2)
memory usage: 3.3+ MB


In [ ]:
GOOD = "Positive"
BAD = "Negative"
NEUTRAL = "Neutral"

In [ ]:
data.loc[data.labels == 'bad', 'labels'] = BAD
data.loc[data.labels == 'good', 'labels'] = GOOD
data.loc[data.labels == 'neutral', 'labels'] = NEUTRAL

In [ ]:
!pip install pycorenlp
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9001')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pycorenlp: filename=pycorenlp-0.3.0-py3-none-any.whl size=2142 sha256=254452ad004dc76e59e25f184f1b70dbcc1864e7e1d2e0c006a79456764dfa59
  Stored in directory: /root/.cache/pip/wheels/99/43/83/53a89b5afcfe4bdaeec4ef0ff54ab0e09d6d3683d2884d1b4d
Successfully built pycorenlp


In [ ]:
def get_sentiment(text):
    output = nlp.annotate(text, properties={
        'annotators': 'sentiment',
        'outputFormat': 'json'
    })
    json_data = json.loads(output)
    sentiment = json_data["sentences"][0]["sentiment"]
    return sentiment

,Sentence,Sentiment
0,This is a positive sentence.,Positive
1,This is a bad day.,Negative
2,This is an apple.,Neutral


In [ ]:
df_1 = data.head(4000)
type(df_1)

pandas.core.frame.DataFrame

In [ ]:
df_1['Sentiment'] = df_1['tweets'].apply(get_sentiment)
df_1.head()

<ipython-input-22-c1e7f8b46838>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['Sentiment'] = df_1['tweets'].apply(get_sentiment)


,tweets,labels,Sentiment
0,ChatGPT: Optimizing Language Models for Dialog...,Neutral,Neutral
1,"Try talking with ChatGPT, our new AI system wh...",Positive,Neutral
2,ChatGPT: Optimizing Language Models for Dialog...,Neutral,Neutral
3,"THRILLED to share that ChatGPT, our new model ...",Positive,Positive
4,"As of 2 minutes ago, @OpenAI released their ne...",Negative,Neutral


In [ ]:
print(df_1.iloc[2,0])

ChatGPT: Optimizing Language Models for Dialogue https://t.co/GLEbMoKN6w #AI #MachineLearning #DataScience #ArtificialIntelligence\n\nTrending AI/ML Article Identified &amp; Digested via Granola; a Machine-Driven RSS Bot by Ramsey Elbasheer https://t.co/RprmAXUp34


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
# Print accuracy score and confusion matrix on test set
print('Accuracy on the test set: ', accuracy_score(df_1['labels'], df_1['Sentiment']))
print(confusion_matrix(df_1['labels'], df_1['Sentiment'])/len(df_1['labels']))

Accuracy on the test set:  0.36175
[[1.1050e-01 3.0250e-01 4.6250e-02 5.0000e-04 1.5000e-03]
 [5.2500e-02 1.2925e-01 8.0000e-02 2.5000e-04 2.5000e-03]
 [4.5500e-02 9.3500e-02 1.2200e-01 0.0000e+00 1.3250e-02]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]]


In [ ]:
# Shut down the background CoreNLP server
client.stop()

time.sleep(10)
!ps -o pid,cmd | grep java

  25834 /bin/bash -c ps -o pid,cmd | grep java
  25836 /bin/bash -c ps -o pid,cmd | grep java
